## Train NLU Model

In [160]:
# Train RASA NLU
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer

training_data = load_data("./CSV Data File/Soum_CSV_Intent_File/Soum_Prepare_JSON_File.json")
trainer = Trainer(RasaNLUConfig("nlu_model_config.json"))
trainer.train(training_data)
model_directory = trainer.persist('models/nlu/', fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.6min finished


## Load Pre-Train Model
#### (If you did not add new data into the training set, please ignore Train NLU Model part and start from here, as it would take you sometime)

In [161]:
# CALL Pre-Train Model
from rasa_nlu.model import Metadata, Interpreter
from rasa_nlu.config import RasaNLUConfig

interpreter = Interpreter.load('./models/nlu/default/current', RasaNLUConfig("./nlu_model_config.json"))

## Test Block for Single Sentence Entity Extraction 

In [169]:
# Test Block for RASA NLU Performance
Sentence="great bye bye"
interpreter.parse(Sentence)

{'entities': [],
 'intent': {'confidence': 0.9852473080854874, 'name': 'Gratitude'},
 'intent_ranking': [{'confidence': 0.9852473080854874, 'name': 'Gratitude'},
  {'confidence': 0.003517996635202802, 'name': 'frustration'},
  {'confidence': 0.0031730979246136614, 'name': 'continue'},
  {'confidence': 0.0030402597926260867, 'name': 'Swear'},
  {'confidence': 0.0016073443695728883, 'name': 'amusement'},
  {'confidence': 0.0005971595665649613, 'name': 'Greeting'},
  {'confidence': 0.0005129755929472183, 'name': 'Restart'},
  {'confidence': 0.000405042705722128, 'name': 'None'},
  {'confidence': 0.0003804103929628181, 'name': 'Anything'},
  {'confidence': 0.00035942057553903434, 'name': 'inform'}],
 'text': 'great bye bye'}

## Dialog Logic and Offer Filter Logic

In [113]:
# Customer input to tell you what they want
def Cust_Raw_Input():
    text = input("Me: ")
    return text

# Capture Entities from what customer say
def Entity_Capture(Customer_input, Customer_Preference):
    import warnings
    warnings.filterwarnings("ignore")
    # Make it lower case so that it is not case sensetive
    Cuisine_List=[x.lower() for x in['Japanese','Asian','Italian','Chinese','Western','Cantonese','Singaporean',
                                     'Thai','Indian','French','Korean','Guangdong','taiwanese','Hong Kong','Bar',
                                     'Mexican','Swedish','Latvian','Spanish','American','Scottish','British',
                                     'Canadian','Russian','Jewish','Polish','German','Hawaiian','Brazilian',
                                     'Peruvian','Salvadorian','Cuban','Tibetan','Egyptian','Greek','Belgian',
                                     'Irish','Welsh','Mormon','Cajun','Portuguese','Turkish','Haitian',
                                     'Tahitian','Kenyan','Algerian','Nigerian','Libyan']]
    Time_List=[x.lower() for x in ['Dinner','Lunch','Afternoon Tea','Breakfast']]
    Location_List=[x.lower() for x in ['Central','Causeway Bay','Tai Kok Tsui']]
    Information_Extraction=interpreter.parse(Customer_input)
    
    if len(Information_Extraction['entities'])==0:
        if any(Information_Extraction["text"].lower() in CLC for CLC in Cuisine_List):
            Customer_Preference["cuisine"]=Information_Extraction['text'].lower()
        elif any(Information_Extraction['text'].lower() in TLC for TLC in Time_List):
            Customer_Preference["time"]["meal"]=Information_Extraction['text'].lower()
        elif any(Information_Extraction['text'].lower() in LLC for LLC in Location_List):
            Customer_Preference["Location"]=Information_Extraction['text'].lower()
        return Customer_Preference
    else:
        # Check if "Not_cuisine" in entity
        NoNotCuisine=True
        for Entity in Information_Extraction['entities']:
            NoNotCuisine = NoNotCuisine and Entity['entity']!="Not_cuisine"
        
        for Entity in Information_Extraction['entities']:
            if type(Entity['value'])==dict:
                ExactTime_known=Customer_Preference["time"]["exact_time"]!="Unknown"
                ExactTime_is_str=type(Customer_Preference["time"]["exact_time"])==str
                if Entity['extractor']=='ner_duckling' and Entity['entity']=='time' and ExactTime_known and ExactTime_is_str:
                    date=Customer_Preference["time"]["exact_time"][0:10]
                    Customer_Preference["time"]["exact_time"]=Entity['value']
                    Customer_Preference["time"]["exact_time"]["from"]=date+Customer_Preference["time"]["exact_time"]["from"][10:]
                    Customer_Preference["time"]["exact_time"]["to"]=date+Customer_Preference["time"]["exact_time"]["to"][10:]
                elif Entity['extractor']=='ner_duckling' and Entity['entity']=='time':
                    Customer_Preference["time"]["exact_time"]=Entity['value']
            elif type(Entity['value'])==str:
                TimeListCheck=any(Entity['value'].lower() in TLC for TLC in Time_List) or any(Entity['entity'].lower() in TLC2 for TLC2 in Time_List)
                CuisineListCheck=any(Entity['value'].lower() in CLC for CLC in Cuisine_List)
                ExactT_known=Customer_Preference["time"]["exact_time"]!="Unknown"
                ExactT_is_dict=type(Customer_Preference["time"]["exact_time"])==dict
                TextNotmay=True
                if Entity['extractor'] == 'ner_duckling':
                    TextNotmay = Entity['text']!='may'
                
                # Here we extract *value as cuisine type, we do not capture cuisine out of our knowledge
                if any(Information_Extraction["text"].lower() in CLC for CLC in Cuisine_List):
                    Customer_Preference["cuisine"]=Information_Extraction['text'].lower()
                elif CuisineListCheck and Entity['extractor']=='ner_crf' and Entity['entity']=='Cuisine':
                    Customer_Preference["cuisine"]=Entity['value'].lower()
                # Exclude ner_spacy for cuisine now as of not_cuisine
                elif CuisineListCheck and Entity['extractor']=='ner_spacy' and NoNotCuisine:
                    Customer_Preference["cuisine"]=Entity['value'].lower()
                
                if any(Information_Extraction['text'].lower() in TLC for TLC in Time_List):
                    Customer_Preference["time"]["meal"]=Information_Extraction['text'].lower()
                elif TimeListCheck and Entity['extractor']=='ner_crf':
                    Customer_Preference["time"]["meal"]=Entity['value'].lower()
                elif TimeListCheck and Entity['extractor']=='ner_spacy':
                    Customer_Preference["time"]["meal"]=Entity['value'].lower()
                #For update exact time when it is already there: if dict
                elif Entity['extractor']=='ner_duckling' and Entity['entity']=='time' and ExactT_known and ExactT_is_dict and TextNotmay:
                    Customer_Preference["time"]["exact_time"]["from"]=Entity['value'][0:10]+Customer_Preference["time"]["exact_time"]["from"][10:]
                    Customer_Preference["time"]["exact_time"]["to"]=Entity['value'][0:10]+Customer_Preference["time"]["exact_time"]["to"][10:]
                elif Entity['extractor']=='ner_duckling' and Entity['entity']=='time' and TextNotmay:
                    Customer_Preference["time"]["exact_time"]=Entity['value']
                    
                # Extract Location
                if any(Information_Extraction['text'].lower() in LLC for LLC in Location_List):
                    Customer_Preference["Location"]=Information_Extraction['text'].lower()
                elif Entity['entity'].lower()=='location' and Entity['extractor']=='ner_crf' and ('processors' not in Entity):
                    # Use value incase it is Lat Long structure
                    Customer_Preference["Location"]=Entity['value'].lower()
                elif Entity['entity'].lower()=='location' and Entity['extractor']=='ner_crf' and ('processors' in Entity):
                    Customer_Preference["Location"]=Information_Extraction['text'][
                        int(Entity['start']):int(Entity['end'])].lower()
                elif Entity['value'].lower()=='location' and Entity['extractor']=='ner_spacy' and ('processors' in Entity):
                    Customer_Preference["Location"]=Information_Extraction['text'][
                        int(Entity['start']):int(Entity['end'])].lower()
                
        return Customer_Preference




In [84]:
# After Capturing all customer needs, filer on the offer list and get recommendations
def Action_Offer_Filter(Customer_Preference,Offer_List):
    import pandas as pd
    from dateutil import parser
    import datetime
    
    CuisineList=['Cuisine Type 1','Cuisine Type 2','Cuisine Type 3','Cuisine Type 4','Cuisine Type 5']
    InvalidDateList=['Invalid Date 1','Invalid Date 2','Invalid Date 3','Invalid Date 4',
                     'Invalid Date 5','Invalid Date 6','Invalid Date 7','Invalid Date 8',
                     'Invalid Date 9','Invalid Date 10','Invalid Date 11','Invalid Date 12',
                     'Invalid Date 13','Invalid Date 14','Invalid Date 15','Invalid Date 16',
                     'Invalid Date 17','Invalid Date 18','Invalid Date 19','Invalid Date 20',
                     'Invalid Date 21','Invalid Date 22','Invalid Date 23','Invalid Date 24',
                     'Invalid Date 25','Invalid Date 26','Invalid Date 27','Invalid Date 28',
                     'Invalid Date 29','Invalid Date 30','Invalid Date 31','Invalid Date 32',
                     'Invalid Date 33','Invalid Date 34','Invalid Date 35','Invalid Date 36',
                     'Invalid Date 37','Invalid Date 38','Invalid Date 39','Invalid Date 40',
                     'Invalid Date 41']
    
    FilterList=[]
    # Filter on Location
    Offer_List=Offer_List.loc[(Offer_List['Landmark'].str.lower() == Customer_Preference['Location'])]
    # Filter on Cuisine
    for CL in CuisineList:
        FilterList.append(Offer_List.loc[(Offer_List[CL].notnull())].loc[
            (Offer_List[CL].str.lower() == Customer_Preference['cuisine'])])
    Offer_List=pd.concat(FilterList)
    Offer_List=Offer_List.drop_duplicates()
    FilterList=[]
    
    if type(Customer_Preference['time']['exact_time'])==dict:
        # In case Date across two day
        if (parser.parse(Customer_Preference['time']['exact_time']['from']).date()!=parser.parse(Customer_Preference['time']['exact_time']['to']).date()) & (parser.parse(Customer_Preference['time']['exact_time']['to']).time()==datetime.time(0, 0)):
            TimeTo=datetime.time(23, 59)
        else:
            TimeTo=parser.parse(Customer_Preference['time']['exact_time']['to']).time()
        
        # Filter on date and time
        for IDL in InvalidDateList:
            FilterList=[]
            FilterList.append(Offer_List.loc[(Offer_List[IDL].isnull())])
            FilterList.append(Offer_List.loc[(Offer_List[IDL].notnull())].loc[
                ((pd.to_datetime(Offer_List[IDL]) != 
                  parser.parse(Customer_Preference['time']['exact_time']['from']).date())
                | (pd.to_datetime(Offer_List[IDL]) != 
                   parser.parse(Customer_Preference['time']['exact_time']['to']).date()))] )
            Offer_List=pd.concat(FilterList)
            
        # Time Filter
        Offer_List=Offer_List.loc[( ((parser.parse(Customer_Preference['time']['exact_time']['from']).time() <=
                            pd.to_datetime(Offer_List['Time range start time']).dt.time) 
                        & (TimeTo >= pd.to_datetime(Offer_List['Time range start time']).dt.time))
                        | ((parser.parse(Customer_Preference['time']['exact_time']['from']).time() <=
                            pd.to_datetime(Offer_List['Time range end time']).dt.time) 
                        & (TimeTo >= pd.to_datetime(Offer_List['Time range end time']).dt.time)) )]
        
        if Customer_Preference['time']['meal']!='Unknown':
            # Filter on Meal when it is not Unknown
            Offer_List=Offer_List.loc[(Offer_List['Meal Type'].str.lower()==Customer_Preference['time']['meal'])]
        
    elif Customer_Preference['time']['exact_time']=='Unknown':
        
        if Customer_Preference['time']['meal']!='Unknown':
            # Filter on Meal when it is not Unknown
            Offer_List=Offer_List.loc[(Offer_List['Meal Type'].str.lower()==Customer_Preference['time']['meal'])]
    elif Customer_Preference['time']['exact_time']!='Unknown':
        # Filter on date and time
        for IDL in InvalidDateList:
            FilterList=[]
            FilterList.append(Offer_List.loc[(Offer_List[IDL].isnull())])
            FilterList.append(Offer_List.loc[(Offer_List[IDL].notnull())].loc[
                (pd.to_datetime(Offer_List[IDL]) != parser.parse(Customer_Preference['time']['exact_time']).date())] )
            Offer_List=pd.concat(FilterList)
        # Filter on time
        if parser.parse(Customer_Preference['time']['exact_time']).time() != datetime.time(0, 0):
            Offer_List=Offer_List.loc[(parser.parse(Customer_Preference['time']['exact_time']).time() >= 
                                        pd.to_datetime(Offer_List['Time range start time']).dt.time)
                                       & (parser.parse(Customer_Preference['time']['exact_time']).time() <= 
                                        pd.to_datetime(Offer_List['Time range end time']).dt.time)]
        
        if Customer_Preference['time']['meal']!='Unknown':
            # Filter on Meal when it is not Unknown
            Offer_List=Offer_List.loc[(Offer_List['Meal Type'].str.lower()==Customer_Preference['time']['meal'])]
    
    filtered=Offer_List.filter(items=['websiteImageUrl','Meal Type','Landmark','Cuisine Type 1',
                                    'Cuisine Type 2','Cuisine Type 3','Cuisine Type 4','Cuisine Type 5'])
    return filtered


In [137]:
def DialogFlow(Customer_Preference={"cuisine": "Unknown", 
                                    "time": {"meal": "Unknown", "exact_time": "Unknown"}, 
                                    "Location": "Unknown"}):
    # Load Offer List for Recommendation Purpose
    import pandas as pd
    import datetime
    Offer_List=pd.read_csv("./CSV Data File/Offer List/Offer List.csv")
    
    print("BOT: To get started just tell me when, where and what you'd like to eat, e.g. 'Asian dining offers this evening in Lockhart Road'.")
    Customer_input=Cust_Raw_Input()
    Customer_Preference=Entity_Capture(Customer_input, Customer_Preference)
    #print(Customer_Preference)
    # fill missing data
    while (Customer_Preference["cuisine"]=="Unknown" or (Customer_Preference["time"]["meal"]=="Unknown" 
                                                         and Customer_Preference["time"]["exact_time"]=="Unknown") 
           or Customer_Preference["Location"]=="Unknown"):
        if Customer_Preference["cuisine"]=="Unknown":
            print("BOT: What kind of cuisine do you prefer?")
            Customer_input=Cust_Raw_Input()
            Customer_Preference=Entity_Capture(Customer_input, Customer_Preference)
        elif Customer_Preference["time"]["meal"]=="Unknown" and Customer_Preference["time"]["exact_time"]=="Unknown":
            print("BOT: When do you want to have it?")
            Customer_input=Cust_Raw_Input()
            Customer_Preference=Entity_Capture(Customer_input, Customer_Preference)
        elif Customer_Preference["Location"]=="Unknown":
            print("BOT: Would you tell me where you are?")
            Customer_input=Cust_Raw_Input()
            Customer_Preference=Entity_Capture(Customer_input, Customer_Preference)
    
    print("Sure, here is what I found: ")
    #print("Pretent That we have done filter:-)")
    print("Filter on (for validation case show as follow): ")
    print(Customer_Preference)
    print(Action_Offer_Filter(Customer_Preference,Offer_List))
    # Action Step on Aciton_Offer_Filter
    print("You will need to pay with a HSBC card :-).")
    return
    

##  Run Your Bot Below

In [139]:
DialogFlow(Customer_Preference={"cuisine": "Unknown", 
                                "time": {"meal": "Unknown", "exact_time": "Unknown"}, 
                                "Location": "Unknown"})

BOT: To get started just tell me when, where and what you'd like to eat, e.g. 'Asian dining offers this evening in Lockhart Road'.
Me: Spanish food is my favourite!
BOT: When do you want to have it?
Me: I'm thinking of late evening as I got to work overtime
BOT: Would you tell me where you are?
Me: Sitting at Central now
Sure, here is what I found: 
Filter on (for validation case show as follow): 
{'cuisine': 'spanish', 'time': {'meal': 'Unknown', 'exact_time': {'to': '2018-03-08T00:00:00.000Z', 'from': '2018-03-08T18:00:00.000Z'}}, 'Location': 'central'}
Empty DataFrame
Columns: [websiteImageUrl, Meal Type, Landmark, Cuisine Type 1, Cuisine Type 2, Cuisine Type 3, Cuisine Type 4, Cuisine Type 5]
Index: []
You will need to pay with a HSBC card :-).
